In [ ]:
!wget https://storage.googleapis.com/ss4-exp-datasource/legalAct/legal-act-classification.zip

--2024-05-09 06:35:25--  https://storage.googleapis.com/ss4-exp-datasource/legalAct/legal-act-classification.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.250.141.207, 142.251.2.207, 2607:f8b0:4023:c0b::cf, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.250.141.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1498270 (1.4M) [application/x-zip-compressed]
Saving to: ‘legal-act-classification.zip’

legal-act-classific 100%[===================>]   1.43M  1.65MB/s    in 0.9s    

2024-05-09 06:35:27 (1.65 MB/s) - ‘legal-act-classification.zip’ saved [1498270/1498270]



In [ ]:
!pip install -q catboost pycaret[full]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.1/486.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.8/21.8 MB 12.5 M

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import

In [ ]:
import zipfile
import os

def extract_zip(zip_file, extract_to):
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"Successfully extracted {zip_file} to {extract_to}")

# Example usage:
zip_file_path = 'legal-act-classification.zip'
extract_to_directory = 'legal-act-classification'

extract_zip(zip_file_path, extract_to_directory)


Successfully extracted legal-act-classification.zip to legal-act-classification


In [ ]:
import pandas as pd
committee = pd.read_csv('legal-act-classification/committee.csv')
patterns = pd.read_csv('legal-act-classification/patterns.csv')
raw = pd.read_csv('legal-act-classification/raw.csv')
train = pd.read_csv('legal-act-classification/train.csv')
test = pd.read_csv('legal-act-classification/test.csv')
sample_submission = pd.read_csv('legal-act-classification/sample_submission.csv')

In [ ]:
!wget -q https://github.com/google/fonts/raw/main/ofl/sarabun/Sarabun-Regular.ttf
import matplotlib as mpl
mpl.font_manager.fontManager.addfont('Sarabun-Regular.ttf')
mpl.rc('font', family='Sarabun')

In [ ]:
raw.head(20)

,rgno,จำนวนประโยค,อำนาจกรรมการต้นฉบับ,มีเงื่อนไข,Subgroupid,การจัดการ,Subgrp_desc,อำนาจกรรมการตาม Subgroup,ข้อความเงื่อนไข
0,105486000056,1,กรรมการอย่างน้อยสองคนมีอำนาจลงลายมือชื่อและประ...,NaN,11007,by system,กรรมการอย่างน้อย x คนลงลายมือชื่อร่วมกันและประ...,กรรมการอย่างน้อยสองคนมีอำนาจลงลายมือชื่อและประ...,NaN
1,105492000372,1,นายเชษฐ์ ทัดทอง หรือนายอรุชา เขมทโรนนท์ หรือ น...,NaN,20012,Manual,(นาย ก. นาย ข. นาย ค….กรรมการ x ใน y) ลงลายมือ...,นายเชษฐ์ ทัดทอง หรือนายอรุชา เขมทโรนนท์ หรือ น...,NaN
2,105493000392,2,กรรมการกลุ่ม เอ ได้แก่ นายธีรเมธ ภู่ทองนายวัชร...,NaN,20023,Manual,(นาย ก. หรือ นาย ข. หรือ นาย ค. หรือ...) ลงลาย...,กรรมการสอง ท่านในกลุ่ม เอ ลงลายมือชื่อร่วมกันแ...,NaN
3,105493000392,2,กรรมการกลุ่ม เอ ได้แก่ นายธีรเมธ ภู่ทองนายวัชร...,Y,20003,Manual,(นาย ก. หรือ นาย ข. หรือ นาย ค. หรือ...) ลงลาย...,กรรมการ หนึ่ง ท่านในกลุ่ม เอลงลายมือชื่อร่วมกั...,กรรมการกลุ่ม เอ ได้แก่ นายอดัม ซิมกิ้นส์นายวิล...
4,105495000276,1,นายไกรเทพ สิริสารวรรณกุล หรือ นายชาญธิษณ์ พัวศ...,NaN,20012,Manual,(นาย ก. นาย ข. นาย ค….กรรมการ x ใน y) ลงลายมือ...,นายไกรเทพ สิริสารวรรณกุล หรือ นายชาญธิษณ์ พัวศ...,NaN
5,105495000519,1,กรรมการอย่างน้อย 2 คน ร่วมกันมีอำนาจลงลายมือชื...,NaN,11007,by system,กรรมการอย่างน้อย x คนลงลายมือชื่อร่วมกันและประ...,กรรมการอย่างน้อย 2 คน ร่วมกันมีอำนาจลงลายมือชื...,NaN
6,105497000788,1,นายธีรดนย์ แก้วนก เเละ นายสุเมธ คุ้มคงลงลายมือ...,NaN,30006,Manual,นาย ก.และ นาย ข.ลงลายมือชื่อร่วมกับ (นาย ค.หรื...,นายธีรดนย์ แก้วนก เเละ นายสุเมธ คุ้มคง ลงลายมื...,NaN
7,105501000333,2,กลุ่ม (ก) 1. นายปฏิภัทธิ์ อุคำ2.นายณัฐกานต์ โค...,NaN,20003,Manual,(นาย ก. หรือ นาย ข. หรือ นาย ค. หรือ...) ลงลาย...,กรรมการคนใดคนหนึ่งของกลุ่ม (ก) ลงลายมือชื่อร่ว...,NaN
8,105501000333,2,กลุ่ม (ก) 1. นายปฏิภัทธิ์ อุคำ2.นายณัฐกานต์ โค...,NaN,20023,Manual,(นาย ก. หรือ นาย ข. หรือ นาย ค. หรือ...) ลงลาย...,กรรมการกลุ่ม (ก) สองคนลงลายมือชื่อร่วมกันและปร...,NaN
9,105503000977,1,นายวชิรวิทย์ เปรมไธสง หรือ นางวชิรวิทย์ เปรมไธ...,NaN,30011,Manual,(นาย ก.หรือ นาย ข. หรือ นาย ค.หรือ...) ลงลายมื...,นายวชิรวิทย์ เปรมไธสง หรือ นางวชิรวิทย์ เปรมไธ...,NaN


In [ ]:
committee.head()

,Unnamed: 0,rgno,line,title,fname,lname
0,0,105484000075,1,นาง,เนติชัย,วงษ์คำภู
1,1,105484000075,2,นาย,ภัทร,ทิพย์อักษร
2,2,105484000075,3,นาย,ธนันทิกานต์,ราชาเดช
3,3,105484000075,4,นาย,นทีธาร,ปัญญากร
4,4,105484000075,5,นางสาว,กนกกฤษฏิ์,กังวาลเกียรติ


In [ ]:
patterns.head(20)

,No,Pattern,Pattern.1
0,1,11001,กรรมการ x คนลงลายมือชื่อ
1,2,11002,กรรมการ x คนลงลายมือชื่อและประทับตราสำคัญของบร...
2,3,11003,กรรมการ x ใน y คนลงลายมือชื่อ
3,4,11004,กรรมการ x ใน y คนลงลายมือชื่อและประทับตราสำคัญ...
4,5,11005,กรรมการ x คนขึ้นไปลงลายมือชื่อและประทับตราของบ...
5,6,11006,กรรมการ x คนขึ้นไปลงลายมือชื่อ
6,7,11007,กรรมการอย่างน้อย x คนลงลายมือชื่อร่วมกันและประ...
7,8,11008,กรรมการอย่างน้อย x คนลงลายมือชื่อร่วมกัน
8,9,11009,กรรมการคนที่ x คนลงลายมือชื่อและประทับตราของบร...
9,10,11010,กรรมการคนที่ x คนลงลายมือชื่อ


In [ ]:
train

,id,rgno,context,pattern,question,legal_act,condition,answer
0,0,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['พศิน บัวขาว'],การทำนิติกรรม สำนักงานตรวจคนเข้าเมือง,NaN,0
1,1,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['นภัสกร แซ่เนี้ยว'],การทำนิติกรรม กรมทางหสวง,NaN,0
2,2,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['ภูวสิษฏ์ วิภาสชีวิน'],อสังหาริมทรัพย์,NaN,0
3,3,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['ธนันทิกานต์ ราชาเดช'],การทำนิติกรรม หน่วยราชการวิสาหกิจ,NaN,0
4,4,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['นรากรณ์ ดีเย็น'],อสังหาริมทรัพย์,NaN,0
...,...,...,...,...,...,...,...,...
4424,4424,1.055260e+11,นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...,20010,"['อานนท์ แซ่อึ่ง', 'ฐีรชัย พรสุริยะศักดิ์', 'ธ...",ใบอนุญาตจำหน่ายสุราและจำหน่ายยาสูบ,NaN,1
4425,4425,1.055260e+11,นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...,20010,"['อานนท์ แซ่อึ่ง', 'อุกฤษฏ์ ดำแดง', 'ธีรพล แสง...",การทำนิติกรรม กรมศุลกากร,NaN,1
4426,4426,1.055260e+11,นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...,20010,"['อุกฤษฏ์ ดำแดง', 'ฐีรชัย พรสุริยะศักดิ์', 'ธี...",การทำนิติกรรมผูกพันส่วนงานราชการ รัฐวิสาหกิจ,NaN,0
4427,4427,1.055260e+11,นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...,20010,"['อานนท์ แซ่อึ่ง', 'ฐีรชัย พรสุริยะศักดิ์', 'ธ...",การทำนิติกรรม การนิคมอุตสาหกรรม,NaN,1


In [ ]:
test

,id,rgno,context,pattern,question,legal_act,condition
0,0,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['สหชา อินทร์ไชย'],การขอวีซ่า,NaN
1,1,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['สุเมธ คุ้มคง'],การค้ำประกัน,NaN
2,2,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['ภควัฒน์ ผาสุข'],ใบอนุญาตจำหน่ายสุราและจำหน่ายยาสูบ,NaN
3,3,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['อชิระ อาจนาเสียว'],ธนาคาร,NaN
4,4,105510003412,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,11015,['ณัฐวีร์ แนวกำพล'],มัคคุเทศน์,NaN
...,...,...,...,...,...,...,...
5830,5830,105562113421,กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...,20004,"['กฤตภาส หาญช้าง', 'กัมปนาท สุนทรธรรม', 'ตุลย์...",ธุรกิจนำเที่ยว,NaN
5831,5831,105562113421,กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...,20004,"['กฤตภาส หาญช้าง', 'พิชญะ แสงรุ่งคงคา', 'ตุลย์...",มัคคุเทศน์,NaN
5832,5832,105562113421,กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...,20004,"['ณัฐกานต์ โคตรยอด', 'กฤตภาส หาญช้าง', 'ตุลย์ ...",การทำนิติกรรม กรมที่ดิน,NaN
5833,5833,105562113421,กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...,20004,"['กฤตภาส หาญช้าง', 'วรนัยน์ นางาซาวา', 'ตุลย์ ...",การทำนิติกรรม สำนักงานส่งเริมการลงทุน,NaN


In [ ]:
train.head()

,id,rgno,context,pattern,question,legal_act,condition,answer
0,0,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['พศิน บัวขาว'],การทำนิติกรรม สำนักงานตรวจคนเข้าเมือง,NaN,0
1,1,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['นภัสกร แซ่เนี้ยว'],การทำนิติกรรม กรมทางหสวง,NaN,0
2,2,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['ภูวสิษฏ์ วิภาสชีวิน'],อสังหาริมทรัพย์,NaN,0
3,3,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['ธนันทิกานต์ ราชาเดช'],การทำนิติกรรม หน่วยราชการวิสาหกิจ,NaN,0
4,4,1.055290e+11,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,11016,['นรากรณ์ ดีเย็น'],อสังหาริมทรัพย์,NaN,0


In [ ]:
y = train['answer']
y

0       0
1       0
2       0
3       0
4       0
       ..
4424    1
4425    1
4426    0
4427    1
4428    0
Name: answer, Length: 4429, dtype: int64

In [ ]:
train.drop(columns=['id','rgno','pattern','condition','answer'], inplace=True)

In [ ]:
train

,context,question,legal_act
0,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,['พศิน บัวขาว'],การทำนิติกรรม สำนักงานตรวจคนเข้าเมือง
1,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,['นภัสกร แซ่เนี้ยว'],การทำนิติกรรม กรมทางหสวง
2,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,['ภูวสิษฏ์ วิภาสชีวิน'],อสังหาริมทรัพย์
3,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,['ธนันทิกานต์ ราชาเดช'],การทำนิติกรรม หน่วยราชการวิสาหกิจ
4,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,['นรากรณ์ ดีเย็น'],อสังหาริมทรัพย์
...,...,...,...
4424,นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...,"['อานนท์ แซ่อึ่ง', 'ฐีรชัย พรสุริยะศักดิ์', 'ธ...",ใบอนุญาตจำหน่ายสุราและจำหน่ายยาสูบ
4425,นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...,"['อานนท์ แซ่อึ่ง', 'อุกฤษฏ์ ดำแดง', 'ธีรพล แสง...",การทำนิติกรรม กรมศุลกากร
4426,นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...,"['อุกฤษฏ์ ดำแดง', 'ฐีรชัย พรสุริยะศักดิ์', 'ธี...",การทำนิติกรรมผูกพันส่วนงานราชการ รัฐวิสาหกิจ
4427,นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...,"['อานนท์ แซ่อึ่ง', 'ฐีรชัย พรสุริยะศักดิ์', 'ธ...",การทำนิติกรรม การนิคมอุตสาหกรรม


In [ ]:
train['concat'] = train.apply(lambda x: ' '.join(x.astype(str)), axis=1)
train.head()

,context,question,legal_act,concat
0,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,['พศิน บัวขาว'],การทำนิติกรรม สำนักงานตรวจคนเข้าเมือง,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
1,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,['นภัสกร แซ่เนี้ยว'],การทำนิติกรรม กรมทางหสวง,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
2,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,['ภูวสิษฏ์ วิภาสชีวิน'],อสังหาริมทรัพย์,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
3,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,['ธนันทิกานต์ ราชาเดช'],การทำนิติกรรม หน่วยราชการวิสาหกิจ,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
4,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...,['นรากรณ์ ดีเย็น'],อสังหาริมทรัพย์,กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...


In [ ]:
test.drop(columns=['id','rgno','pattern'], inplace=True)

In [ ]:
test['concat'] = test.apply(lambda x: ' '.join(x.astype(str)), axis=1)
test.head()

,context,question,legal_act,condition,concat
0,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,['สหชา อินทร์ไชย'],การขอวีซ่า,NaN,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
1,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,['สุเมธ คุ้มคง'],การค้ำประกัน,NaN,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
2,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,['ภควัฒน์ ผาสุข'],ใบอนุญาตจำหน่ายสุราและจำหน่ายยาสูบ,NaN,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
3,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,['อชิระ อาจนาเสียว'],ธนาคาร,NaN,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
4,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...,['ณัฐวีร์ แนวกำพล'],มัคคุเทศน์,NaN,ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...


In [ ]:
X = train['concat']
X

0       กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
1       กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
2       กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
3       กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
4       กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
                              ...                        
4424    นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...
4425    นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...
4426    นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...
4427    นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...
4428    นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...
Name: concat, Length: 4429, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=888)

In [ ]:
def preprocess_function(examples):
    inputs = tokenizer(
        X_train,
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    inputs["labels"] = examples["answer"]
    return inputs

tokenized_train = train.map(preprocess_function, batched=True, remove_columns=train.column_names)
tokenized_validation = validation.map(preprocess_function, batched=True, remove_columns=validation.column_names)
tokenized_train

In [ ]:
X_test = test['concat']
X_test

0       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
1       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
2       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
3       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
4       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
                              ...                        
5830    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5831    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5832    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5833    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5834    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
Name: concat, Length: 5835, dtype: object

In [ ]:
# from transformers import AutoTokenizer, AutoModelForSequenceClassification

# tokenizer = AutoTokenizer.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")
# model = AutoModelForSequenceClassification.from_pretrained("airesearch/wangchanberta-base-att-spm-uncased")

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at airesearch/wangchanberta-base-att-spm-uncased and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# tokenized_texts = X_train.apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# # Print the tokenized texts
# tokenized_texts

2151    [5, 331, 7636, 3974, 2271, 9126, 10, 43, 7110,...
1613    [5, 331, 14013, 7912, 10, 3684, 13934, 2259, 2...
429     [5, 331, 3770, 10, 4222, 3383, 10, 31, 10, 198...
772     [5, 10, 10471, 3770, 10230, 11003, 2857, 86, 1...
3389    [5, 10, 994, 1531, 2356, 154, 23597, 129, 2474...
                              ...                        
2748    [5, 10, 6128, 2956, 13969, 635, 10, 5520, 323,...
814     [5, 2956, 13969, 635, 10, 5520, 323, 2377, 331...
1263    [5, 2956, 15306, 9, 10, 864, 470, 10, 154, 235...
1946    [5, 331, 14147, 4165, 10, 22976, 154, 23597, 8...
2558    [5, 10, 994, 171, 10, 61, 11, 10, 15544, 222, ...
Name: concat, Length: 3543, dtype: object

In [ ]:
# val_tokenized_texts = X_val.apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# # Print the tokenized texts
# val_tokenized_texts

1574    [5, 331, 282, 104, 19109, 10, 3811, 417, 3079,...
2608    [5, 1241, 61, 29, 2956, 21463, 56, 417, 13535,...
3268    [5, 10, 994, 1531, 2356, 1496, 154, 23597, 13,...
3200    [5, 10, 994, 291, 20, 2728, 1247, 154, 23597, ...
1760    [5, 331, 20159, 669, 20504, 10, 149, 13070, 10...
                              ...                        
2253    [5, 10, 6678, 10, 994, 2356, 154, 23597, 606, ...
669     [5, 10, 994, 21970, 191, 5668, 2168, 10, 16259...
3188    [5, 10, 994, 291, 20, 17738, 10, 154, 23597, 6...
339     [5, 331, 14147, 4165, 10, 22976, 10, 31, 331, ...
210     [5, 10, 994, 39, 24559, 154, 23597, 899, 994, ...
Name: concat, Length: 886, dtype: object

In [ ]:
# class CustomDataCollator:
#     def __init__(self, X, y):
#         self.X = X
#         self.y = y

#     def __call__(self, batch):
#         # Create tensors for input_ids and labels
#         input_ids = torch.tensor([item for sublist in self.X[batch] for item in sublist])
#         labels = torch.tensor([self.y[batch]])

#         return {"input_ids": input_ids, "labels": labels}

In [ ]:
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="exp_number1",
#     evaluation_strategy="epoch",
#     learning_rate=1e-5,
#     per_device_train_batch_size=16,
#     per_device_eval_batch_size=8,
#     num_train_epochs=10,
#     weight_decay=0.01,
#     push_to_hub=False,
#     logging_dir="logs",
#     logging_steps=10,
# )

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=tokenized_texts,
#     eval_dataset=tokenized_validation,
#     tokenizer=tokenizer,
#     data_collator=data_collator,
# )

# trainer.train()

In [ ]:
!pip install pythainlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 45.7 MB/s eta 0:00:00


In [ ]:
from pythainlp.tokenize import word_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenize)

# Fit and transform the processed text on training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X)

# Transform the processed text on testing data
X_test_tfidf = tfidf_vectorizer.transform(X_test)

X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# แปลง Sparse Matrix เป็น DataFrame
X_test_tfidf_df = pd.DataFrame(X_test_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
X

0       กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
1       กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
2       กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
3       กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
4       กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
                              ...                        
4424    นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...
4425    นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...
4426    นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...
4427    นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...
4428    นายอุกฤษฏ์ ดำแดง หรือนายฐีรชัย พรสุริยะศักดิ์ ...
Name: concat, Length: 4429, dtype: object

In [ ]:
X_test

0       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
1       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
2       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
3       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
4       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
                              ...                        
5830    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5831    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5832    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5833    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5834    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
Name: concat, Length: 5835, dtype: object

In [ ]:
X_merged = pd.concat([X, X_test], ignore_index=True)

In [ ]:
X_merged

0        กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
1        กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
2        กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
3        กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
4        กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
                               ...                        
10259    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
10260    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
10261    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
10262    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
10263    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
Name: concat, Length: 10264, dtype: object

In [ ]:
X_merged.iloc[1:5]

1    กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
2    กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
3    กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
4    กรรมการคนใดคนหนึ่งลงลายมือชื่อร่วมกับกรรมการอื...
Name: concat, dtype: object

In [ ]:
# Concat -> transform -> unmerge GG

from pythainlp.tokenize import word_tokenize
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Initialize the TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(tokenizer=word_tokenize)

# # Fit and transform the processed text on training data
# X_train_tfidf = tfidf_vectorizer.fit_transform(X)

# # Transform the processed text on testing data
# X_test_tfidf = tfidf_vectorizer.transform(X_test)

X_merged_tfidf = tfidf_vectorizer.fit_transform(X_merged)
X_train_tfidf = X_merged_tfidf[:4429]
X_test_tfidf = X_merged_tfidf[4429:]

X_train_tfidf_df = pd.DataFrame(X_train_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

# แปลง Sparse Matrix เป็น DataFrame
X_test_tfidf_df = pd.DataFrame(X_test_tfidf.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
X_train_tfidf_df

,,,,',"',","','",'],(,(1),(2),...,ไพบูลย์,ไฟฟ้า,ไมตรี,ไมเคิล,ไม่,ไม่ต้อง,ไว้,ๆ,่,้
0,0.309297,0.0,0.0,0.000000,0.000000,0.0,0.077324,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.310170,0.0,0.0,0.000000,0.000000,0.0,0.077543,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.241790,0.0,0.0,0.000000,0.000000,0.0,0.080597,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.289655,0.0,0.0,0.000000,0.000000,0.0,0.072414,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.235987,0.0,0.0,0.000000,0.000000,0.0,0.078662,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,0.560309,0.0,0.0,0.060053,0.060053,0.0,0.025469,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4425,0.598199,0.0,0.0,0.061326,0.061326,0.0,0.026009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4426,0.589124,0.0,0.0,0.060396,0.060396,0.0,0.025614,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4427,0.591265,0.0,0.0,0.060615,0.060615,0.0,0.025707,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_test_tfidf_df

,,,,',"',","','",'],(,(1),(2),...,ไพบูลย์,ไฟฟ้า,ไมตรี,ไมเคิล,ไม่,ไม่ต้อง,ไว้,ๆ,่,้
0,0.296578,0.0,0.0,0.000000,0.000000,0.0,0.059316,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.326364,0.0,0.0,0.000000,0.000000,0.0,0.065273,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.269475,0.0,0.0,0.000000,0.000000,0.0,0.053895,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.305606,0.0,0.0,0.000000,0.000000,0.0,0.061121,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.312783,0.0,0.0,0.000000,0.000000,0.0,0.062557,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5830,0.678052,0.0,0.0,0.042073,0.042073,0.0,0.017843,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5831,0.676372,0.0,0.0,0.041969,0.041969,0.0,0.017799,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5832,0.690373,0.0,0.0,0.041739,0.041739,0.0,0.017702,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5833,0.676543,0.0,0.0,0.040903,0.040903,0.0,0.017347,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# old version
X_test_tfidf_df

,,,,',"',","','",'],(,(1),(2),...,ไทย,ไธสง,ไพบูลย์,ไฟฟ้า,ไมเคิล,ไม่,ไม่ต้อง,ไว้,ๆ,่
0,0.270059,0.0,0.0,0.000000,0.000000,0.0,0.054012,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.285414,0.0,0.0,0.000000,0.000000,0.0,0.057083,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.250965,0.0,0.0,0.000000,0.000000,0.0,0.050193,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.269992,0.0,0.0,0.000000,0.000000,0.0,0.053998,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.278721,0.0,0.0,0.000000,0.000000,0.0,0.055744,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5830,0.569205,0.0,0.0,0.035801,0.035801,0.0,0.014979,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5831,0.573149,0.0,0.0,0.036049,0.036049,0.0,0.015083,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5832,0.597237,0.0,0.0,0.036601,0.036601,0.0,0.015314,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5833,0.586174,0.0,0.0,0.035923,0.035923,0.0,0.015030,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
X_train_tfidf_df['y'] = y

In [ ]:
X_train_tfidf_df

,,,,',"',","','",'],(,(1),(2),...,ไฟฟ้า,ไมตรี,ไมเคิล,ไม่,ไม่ต้อง,ไว้,ๆ,่,้,y
0,0.309297,0.0,0.0,0.000000,0.000000,0.0,0.077324,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.310170,0.0,0.0,0.000000,0.000000,0.0,0.077543,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.241790,0.0,0.0,0.000000,0.000000,0.0,0.080597,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.289655,0.0,0.0,0.000000,0.000000,0.0,0.072414,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.235987,0.0,0.0,0.000000,0.000000,0.0,0.078662,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,0.560309,0.0,0.0,0.060053,0.060053,0.0,0.025469,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4425,0.598199,0.0,0.0,0.061326,0.061326,0.0,0.026009,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4426,0.589124,0.0,0.0,0.060396,0.060396,0.0,0.025614,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4427,0.591265,0.0,0.0,0.060615,0.060615,0.0,0.025707,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
# cat_columns = list(X_train_tfidf_df.columns)

In [ ]:
!pwd

/content


In [ ]:
%cp /content/drive/MyDrive/embedding.csv .

In [ ]:
%cp /content/drive/MyDrive/test_embedding.csv .

In [ ]:
cat_columns.remove('y')

In [ ]:
cat_columns[-1]

'่'

In [ ]:
print(cat_columns)

[' ', '  ', '   ', "'", "',", "','", "']", '(', '(1)', '(2)', '(3)', '(4)', '(5)', ')', ')2.', ',', '-', '.', '1', '1,000,000', '10', '10,000,000', '100,000', '2', '2,000,000', '20,000', '20,000,000', '200,000', '2000000', '2545', '3', '3,000,000', '300,000', '4', '5', '5,000,000', '50,000,000', '500,000', '6', '7', '70,000,000', '8', '9', ':', "['", 'nan', 'ก', 'ก.', 'กก', 'กง', 'กน', 'กนก', 'กมล', 'กร', 'กรณี', 'กรณ์', 'กรม', 'กรมการค้าต่างประเทศ', 'กรมการจัดหางาน', 'กรมการปกครอง', 'กรมที่ดิน', 'กรมปศุสัตว์', 'กรมพัฒนาฝีมือแรงงาน', 'กรมวิชาการเกษตร', 'กรมวิทยาศาสตร์การแพทย์', 'กรมศุลกากร', 'กรมสรรพสามิต', 'กรมสรรพากร', 'กรมโรงงานอุตสาหกรรม', 'กรร', 'กรรม', 'กรรมการ', 'กระจาย', 'กระทรวง', 'กระทรวงการต่างประเทศ', 'กระทรวงพาณิชย์', 'กระทรวงมหาดไทย', 'กระทรวงสาธารณสุข', 'กระทรวงอุตสาหกรรม', 'กระทำการ', 'กระบวน', 'กรุงเทพ', 'กรุงเทพมหานคร', 'กฤต', 'กลุ่ม', 'กล่าวคือ', 'กล่าวโทษ', 'กว่า', 'กอง', 'กองทุนประกันสังคม', 'กองบัญชาการ', 'กังวาล', 'กันต์', 'กับ', 'กัมปนาท', 'กานต์', 'การ', 'การกร

In [ ]:
dense_embedding = pd.read_csv("/content/drive/MyDrive/mLUKE_train_embedding_original_ds_fixed_tokenization.csv")

In [ ]:
dense_embedding

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,0.730174,1.202799,0.335345,-0.216292,-1.387141,0.060743,0.275650,0.545680,-0.574699,0.216123,...,1.021036,0.080836,0.167220,-1.092232,0.261472,-0.853913,1.336742,-0.709309,0.936049,-1.225479
1,0.812005,1.545282,0.046956,-0.243333,-1.487650,0.228674,0.220791,0.402795,-0.349810,0.232521,...,1.114909,0.152365,0.050509,-0.898874,0.230370,-0.913556,1.742268,-0.656205,0.984388,-1.038547
2,0.644363,1.228564,0.135615,-0.080580,-1.321632,0.058628,0.288470,0.375576,-0.420645,0.107971,...,1.187575,-0.185193,0.108397,-1.018919,0.208262,-0.986704,1.543338,-0.747534,0.897429,-1.382525
3,0.807354,1.329873,0.191466,-0.223989,-1.334806,0.149881,0.112342,0.512161,-0.372319,-0.016115,...,1.229239,0.135120,0.184400,-0.905826,0.253566,-1.102298,2.183895,-0.548954,0.990621,-1.152954
4,0.913339,1.039073,0.243332,-0.014952,-1.110743,0.082357,0.020789,0.305253,-0.600671,-0.045323,...,1.096302,-0.005344,0.173958,-0.706796,0.289980,-1.093470,2.030743,-0.579019,1.046302,-1.108032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,0.602108,-1.055011,-0.147303,-0.610011,1.308651,-0.691732,-0.338521,-0.218589,-0.279856,0.020275,...,-0.503790,-0.023859,0.275732,0.128200,0.351783,-0.119606,-0.934477,-0.635799,-0.970571,-0.162488
4425,0.606525,-0.892297,-0.140089,-0.716445,1.486970,-0.839968,-0.386833,-0.323157,-0.340386,-0.056872,...,-0.432943,0.140069,0.300638,0.049395,0.335043,-0.082425,-1.013270,-0.509029,-0.978415,-0.366985
4426,0.572948,1.530818,0.057113,-0.216966,-1.037537,0.346456,0.175458,0.337024,-0.485699,0.556204,...,1.095925,-0.039449,-0.111128,-0.632567,0.055818,-1.061177,1.607499,-0.380530,0.786389,-0.760507
4427,0.602108,-1.055011,-0.147303,-0.610011,1.308651,-0.691732,-0.338521,-0.218589,-0.279856,0.020275,...,-0.503790,-0.023859,0.275732,0.128200,0.351783,-0.119606,-0.934477,-0.635799,-0.970571,-0.162488


In [ ]:
test_dense_embedding = pd.read_csv("/content/drive/MyDrive/mLUKE_test_embedding_original_ds_fixed_tokenization.csv")

In [ ]:
test_dense_embedding

,embedding_0,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,-0.005835,-1.611130,-0.694969,-0.241743,0.794428,-0.631080,-0.308012,-0.610526,-0.018604,-0.103792,...,-0.306825,-0.519683,0.379545,-0.283006,0.357196,0.228782,-0.354101,-0.984394,-0.278065,-0.362803
1,-0.055450,-1.575956,-0.644559,-0.215254,0.811141,-0.645420,-0.346850,-0.619221,-0.042204,-0.064809,...,-0.331662,-0.572508,0.297412,-0.339836,0.357267,0.307861,-0.398132,-0.979745,-0.319100,-0.379158
2,0.030653,-1.602337,-0.660458,-0.302823,0.850932,-0.670356,-0.300819,-0.614479,0.026027,-0.078474,...,-0.292819,-0.545065,0.304475,-0.277811,0.367594,0.227220,-0.422006,-0.971093,-0.335331,-0.347247
3,0.013070,-1.582572,-0.642479,-0.286348,0.828568,-0.664857,-0.350813,-0.665868,0.029208,-0.017508,...,-0.349620,-0.550715,0.359235,-0.298945,0.365292,0.269076,-0.355002,-1.035484,-0.339808,-0.399286
4,-0.006622,-1.577025,-0.609773,-0.236258,0.837385,-0.691014,-0.312239,-0.659146,-0.001464,-0.084743,...,-0.301880,-0.578666,0.330456,-0.343431,0.343656,0.289055,-0.427228,-0.946517,-0.306729,-0.348914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5830,0.361504,1.507129,0.458661,-0.247297,-0.796973,0.073164,0.281467,0.269206,-0.123233,0.228069,...,0.953505,0.195646,0.072143,-0.479258,0.201077,-0.423784,1.122450,-0.541395,0.532338,-0.342229
5831,0.368825,1.580680,0.910092,-0.177114,-0.547772,-0.232202,0.248162,-0.135688,0.069797,0.293301,...,1.081611,-0.147295,0.303330,-0.656561,0.181175,-0.330183,1.068016,-0.596532,0.441062,-0.197209
5832,0.278888,1.445282,0.943363,-0.448431,-0.202601,-0.577712,0.098708,-0.323422,0.003533,0.359469,...,1.157296,-0.188062,0.401843,-0.879180,0.293243,-0.096989,0.640071,-0.889044,0.224857,-0.274063
5833,0.357488,1.588688,0.668592,-0.273153,-0.651621,-0.108377,0.270316,-0.002147,-0.028951,0.315971,...,1.152986,0.140404,0.306716,-0.548324,0.373770,-0.424699,0.984228,-0.491309,0.491280,-0.400934


In [ ]:
Xx = pd.merge(X_train_tfidf_df, dense_embedding, left_index=True, right_index=True)

In [ ]:
Xx

,,,,',"',","','",'],(,(1),(2),...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,0.309297,0.0,0.0,0.000000,0.000000,0.0,0.077324,0.0,0.0,0.0,...,1.021036,0.080836,0.167220,-1.092232,0.261472,-0.853913,1.336742,-0.709309,0.936049,-1.225479
1,0.310170,0.0,0.0,0.000000,0.000000,0.0,0.077543,0.0,0.0,0.0,...,1.114909,0.152365,0.050509,-0.898874,0.230370,-0.913556,1.742268,-0.656205,0.984388,-1.038547
2,0.241790,0.0,0.0,0.000000,0.000000,0.0,0.080597,0.0,0.0,0.0,...,1.187575,-0.185193,0.108397,-1.018919,0.208262,-0.986704,1.543338,-0.747534,0.897429,-1.382525
3,0.289655,0.0,0.0,0.000000,0.000000,0.0,0.072414,0.0,0.0,0.0,...,1.229239,0.135120,0.184400,-0.905826,0.253566,-1.102298,2.183895,-0.548954,0.990621,-1.152954
4,0.235987,0.0,0.0,0.000000,0.000000,0.0,0.078662,0.0,0.0,0.0,...,1.096302,-0.005344,0.173958,-0.706796,0.289980,-1.093470,2.030743,-0.579019,1.046302,-1.108032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,0.560309,0.0,0.0,0.060053,0.060053,0.0,0.025469,0.0,0.0,0.0,...,-0.503790,-0.023859,0.275732,0.128200,0.351783,-0.119606,-0.934477,-0.635799,-0.970571,-0.162488
4425,0.598199,0.0,0.0,0.061326,0.061326,0.0,0.026009,0.0,0.0,0.0,...,-0.432943,0.140069,0.300638,0.049395,0.335043,-0.082425,-1.013270,-0.509029,-0.978415,-0.366985
4426,0.589124,0.0,0.0,0.060396,0.060396,0.0,0.025614,0.0,0.0,0.0,...,1.095925,-0.039449,-0.111128,-0.632567,0.055818,-1.061177,1.607499,-0.380530,0.786389,-0.760507
4427,0.591265,0.0,0.0,0.060615,0.060615,0.0,0.025707,0.0,0.0,0.0,...,-0.503790,-0.023859,0.275732,0.128200,0.351783,-0.119606,-0.934477,-0.635799,-0.970571,-0.162488


In [ ]:
print(Xx.columns)

Index([' ', '  ', '   ', ''', '',', '','', '']', '(', '(1)', '(2)',
       ...
       'embedding_758', 'embedding_759', 'embedding_760', 'embedding_761',
       'embedding_762', 'embedding_763', 'embedding_764', 'embedding_765',
       'embedding_766', 'embedding_767'],
      dtype='object', length=2268)


In [ ]:
y

0       0
1       0
2       0
3       0
4       0
       ..
4424    1
4425    1
4426    0
4427    1
4428    0
Name: answer, Length: 4429, dtype: int64

In [ ]:
Yy = pd.merge(X_test_tfidf_df, test_dense_embedding, left_index=True, right_index=True)

In [ ]:
Yy

,,,,',"',","','",'],(,(1),(2),...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,0.296578,0.0,0.0,0.000000,0.000000,0.0,0.059316,0.0,0.0,0.0,...,-0.306825,-0.519683,0.379545,-0.283006,0.357196,0.228782,-0.354101,-0.984394,-0.278065,-0.362803
1,0.326364,0.0,0.0,0.000000,0.000000,0.0,0.065273,0.0,0.0,0.0,...,-0.331662,-0.572508,0.297412,-0.339836,0.357267,0.307861,-0.398132,-0.979745,-0.319100,-0.379158
2,0.269475,0.0,0.0,0.000000,0.000000,0.0,0.053895,0.0,0.0,0.0,...,-0.292819,-0.545065,0.304475,-0.277811,0.367594,0.227220,-0.422006,-0.971093,-0.335331,-0.347247
3,0.305606,0.0,0.0,0.000000,0.000000,0.0,0.061121,0.0,0.0,0.0,...,-0.349620,-0.550715,0.359235,-0.298945,0.365292,0.269076,-0.355002,-1.035484,-0.339808,-0.399286
4,0.312783,0.0,0.0,0.000000,0.000000,0.0,0.062557,0.0,0.0,0.0,...,-0.301880,-0.578666,0.330456,-0.343431,0.343656,0.289055,-0.427228,-0.946517,-0.306729,-0.348914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5830,0.678052,0.0,0.0,0.042073,0.042073,0.0,0.017843,0.0,0.0,0.0,...,0.953505,0.195646,0.072143,-0.479258,0.201077,-0.423784,1.122450,-0.541395,0.532338,-0.342229
5831,0.676372,0.0,0.0,0.041969,0.041969,0.0,0.017799,0.0,0.0,0.0,...,1.081611,-0.147295,0.303330,-0.656561,0.181175,-0.330183,1.068016,-0.596532,0.441062,-0.197209
5832,0.690373,0.0,0.0,0.041739,0.041739,0.0,0.017702,0.0,0.0,0.0,...,1.157296,-0.188062,0.401843,-0.879180,0.293243,-0.096989,0.640071,-0.889044,0.224857,-0.274063
5833,0.676543,0.0,0.0,0.040903,0.040903,0.0,0.017347,0.0,0.0,0.0,...,1.152986,0.140404,0.306716,-0.548324,0.373770,-0.424699,0.984228,-0.491309,0.491280,-0.400934


# Catboost

In [ ]:
!pip install catboost

In [ ]:
from catboost import CatBoostClassifier, Pool

In [ ]:
model = CatBoostClassifier(verbose=True)

In [ ]:
Xx = Xx.drop("y",  axis='columns')

In [ ]:
Xx

,,,,',"',","','",'],(,(1),(2),...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,0.309297,0.0,0.0,0.000000,0.000000,0.0,0.077324,0.0,0.0,0.0,...,1.021036,0.080836,0.167220,-1.092232,0.261472,-0.853913,1.336742,-0.709309,0.936049,-1.225479
1,0.310170,0.0,0.0,0.000000,0.000000,0.0,0.077543,0.0,0.0,0.0,...,1.114909,0.152365,0.050509,-0.898874,0.230370,-0.913556,1.742268,-0.656205,0.984388,-1.038547
2,0.241790,0.0,0.0,0.000000,0.000000,0.0,0.080597,0.0,0.0,0.0,...,1.187575,-0.185193,0.108397,-1.018919,0.208262,-0.986704,1.543338,-0.747534,0.897429,-1.382525
3,0.289655,0.0,0.0,0.000000,0.000000,0.0,0.072414,0.0,0.0,0.0,...,1.229239,0.135120,0.184400,-0.905826,0.253566,-1.102298,2.183895,-0.548954,0.990621,-1.152954
4,0.235987,0.0,0.0,0.000000,0.000000,0.0,0.078662,0.0,0.0,0.0,...,1.096302,-0.005344,0.173958,-0.706796,0.289980,-1.093470,2.030743,-0.579019,1.046302,-1.108032
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,0.560309,0.0,0.0,0.060053,0.060053,0.0,0.025469,0.0,0.0,0.0,...,-0.503790,-0.023859,0.275732,0.128200,0.351783,-0.119606,-0.934477,-0.635799,-0.970571,-0.162488
4425,0.598199,0.0,0.0,0.061326,0.061326,0.0,0.026009,0.0,0.0,0.0,...,-0.432943,0.140069,0.300638,0.049395,0.335043,-0.082425,-1.013270,-0.509029,-0.978415,-0.366985
4426,0.589124,0.0,0.0,0.060396,0.060396,0.0,0.025614,0.0,0.0,0.0,...,1.095925,-0.039449,-0.111128,-0.632567,0.055818,-1.061177,1.607499,-0.380530,0.786389,-0.760507
4427,0.591265,0.0,0.0,0.060615,0.060615,0.0,0.025707,0.0,0.0,0.0,...,-0.503790,-0.023859,0.275732,0.128200,0.351783,-0.119606,-0.934477,-0.635799,-0.970571,-0.162488


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xx, y, test_size=0.2, random_state=42)

In [ ]:
y_train

1001    0
1345    0
3129    1
3352    1
52      1
       ..
3444    0
466     1
3092    1
3772    0
860     0
Name: answer, Length: 3543, dtype: int64

In [ ]:
X_train

,,,,',"',","','",'],(,(1),(2),...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
1001,0.585939,0.000000,0.0,0.060069,0.060069,0.0,0.025476,0.0,0.0,0.0,...,1.090217,0.153940,-0.004408,-0.664406,0.011791,-0.695944,1.823227,-0.619691,0.713136,-0.657964
1345,0.567029,0.000000,0.0,0.058131,0.058131,0.0,0.024653,0.0,0.0,0.0,...,1.288494,-0.006785,-0.032619,-0.860106,-0.121109,-0.737781,1.785040,-0.583745,0.776953,-0.713169
3129,0.405023,0.000000,0.0,0.119376,0.119376,0.0,0.050628,0.0,0.0,0.0,...,-0.606411,-0.160253,-0.116271,-0.161157,0.352823,0.310079,-0.747155,-0.410271,-0.667796,0.128352
3352,0.321106,0.000000,0.0,0.063095,0.063095,0.0,0.053518,0.0,0.0,0.0,...,-0.697170,0.004392,0.299357,0.199561,0.614878,0.367047,-0.810212,-0.604707,-0.523858,-0.106859
52,0.404798,0.000000,0.0,0.136354,0.136354,0.0,0.057828,0.0,0.0,0.0,...,-0.677607,-0.279613,0.464985,-0.175003,0.337357,0.193812,-0.988146,-0.700999,-0.614881,-0.453688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3444,0.381671,0.000000,0.0,0.056247,0.056247,0.0,0.047709,0.0,0.0,0.0,...,1.134258,0.194426,-0.156864,-0.583341,-0.020413,-0.807282,1.863284,-0.457385,0.773765,-0.662111
466,0.599428,0.114143,0.0,0.029446,0.029446,0.0,0.024976,0.0,0.0,0.0,...,-0.017948,-0.908020,0.133113,-0.251098,0.168397,0.055813,-0.459535,-0.886227,-0.746095,0.360486
3092,0.352752,0.000000,0.0,0.069313,0.069313,0.0,0.058792,0.0,0.0,0.0,...,-0.527853,-0.241808,0.063609,-0.001388,0.465674,0.431004,-0.980584,-0.531457,-0.567712,-0.093955
3772,0.420269,0.000000,0.0,0.110106,0.110106,0.0,0.046697,0.0,0.0,0.0,...,1.126442,0.139496,-0.091322,-0.650269,0.001908,-0.650824,1.963760,-0.619348,0.681922,-0.650349


In [ ]:
model.fit(X_train,y_train, eval_set = (X_test,y_test))

Learning rate set to 0.043312
0:	learn: 0.6017172	test: 0.6014996	best: 0.6014996 (0)	total: 589ms	remaining: 9m 48s
1:	learn: 0.5212672	test: 0.5224193	best: 0.5224193 (1)	total: 883ms	remaining: 7m 20s
2:	learn: 0.4542566	test: 0.4558642	best: 0.4558642 (2)	total: 1.19s	remaining: 6m 35s
3:	learn: 0.4026981	test: 0.4052276	best: 0.4052276 (3)	total: 1.48s	remaining: 6m 8s
4:	learn: 0.3558965	test: 0.3590307	best: 0.3590307 (4)	total: 1.78s	remaining: 5m 54s
5:	learn: 0.3152869	test: 0.3183059	best: 0.3183059 (5)	total: 2.08s	remaining: 5m 44s
6:	learn: 0.2818337	test: 0.2853696	best: 0.2853696 (6)	total: 2.37s	remaining: 5m 36s
7:	learn: 0.2550323	test: 0.2589887	best: 0.2589887 (7)	total: 2.66s	remaining: 5m 30s
8:	learn: 0.2265383	test: 0.2301331	best: 0.2301331 (8)	total: 2.96s	remaining: 5m 25s
9:	learn: 0.2064833	test: 0.2100014	best: 0.2100014 (9)	total: 3.26s	remaining: 5m 22s
10:	learn: 0.1866485	test: 0.1902858	best: 0.1902858 (10)	total: 3.55s	remaining: 5m 18s
11:	learn: 0

In [ ]:
test_pool = Pool(X_test, label=y_test)

In [ ]:
metrics = model.eval_metrics(test_pool,
                             metrics=['F1'],
                             plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [ ]:
metrics

{'F1': [0.9572649572649573,
  0.9572649572649573,
  0.9638554216867469,
  0.9638554216867469,
  0.9638554216867469,
  0.9638554216867469,
  0.967409948542024,
  0.9638554216867469,
  0.9656357388316151,
  0.9638554216867469,
  0.9656357388316151,
  0.9638554216867469,
  0.9656357388316151,
  0.967409948542024,
  0.967409948542024,
  0.9641025641025641,
  0.9641025641025641,
  0.9658703071672354,
  0.9675213675213676,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9642248722316865,
  0.9642248722316865,
  0.9642248722316865,
  0.9642248722316865,
  0.9658703071672354,
  0.9658703071672354,
  0.9641025641025641,
  0.9658703071672354,
  0.9641025641025641,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.9658703071672354,
  0.965

## Catboost inference

In [ ]:
Yy

,,,,',"',","','",'],(,(1),(2),...,embedding_758,embedding_759,embedding_760,embedding_761,embedding_762,embedding_763,embedding_764,embedding_765,embedding_766,embedding_767
0,0.296578,0.0,0.0,0.000000,0.000000,0.0,0.059316,0.0,0.0,0.0,...,-0.306825,-0.519683,0.379545,-0.283006,0.357196,0.228782,-0.354101,-0.984394,-0.278065,-0.362803
1,0.326364,0.0,0.0,0.000000,0.000000,0.0,0.065273,0.0,0.0,0.0,...,-0.331662,-0.572508,0.297412,-0.339836,0.357267,0.307861,-0.398132,-0.979745,-0.319100,-0.379158
2,0.269475,0.0,0.0,0.000000,0.000000,0.0,0.053895,0.0,0.0,0.0,...,-0.292819,-0.545065,0.304475,-0.277811,0.367594,0.227220,-0.422006,-0.971093,-0.335331,-0.347247
3,0.305606,0.0,0.0,0.000000,0.000000,0.0,0.061121,0.0,0.0,0.0,...,-0.349620,-0.550715,0.359235,-0.298945,0.365292,0.269076,-0.355002,-1.035484,-0.339808,-0.399286
4,0.312783,0.0,0.0,0.000000,0.000000,0.0,0.062557,0.0,0.0,0.0,...,-0.301880,-0.578666,0.330456,-0.343431,0.343656,0.289055,-0.427228,-0.946517,-0.306729,-0.348914
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5830,0.678052,0.0,0.0,0.042073,0.042073,0.0,0.017843,0.0,0.0,0.0,...,0.953505,0.195646,0.072143,-0.479258,0.201077,-0.423784,1.122450,-0.541395,0.532338,-0.342229
5831,0.676372,0.0,0.0,0.041969,0.041969,0.0,0.017799,0.0,0.0,0.0,...,1.081611,-0.147295,0.303330,-0.656561,0.181175,-0.330183,1.068016,-0.596532,0.441062,-0.197209
5832,0.690373,0.0,0.0,0.041739,0.041739,0.0,0.017702,0.0,0.0,0.0,...,1.157296,-0.188062,0.401843,-0.879180,0.293243,-0.096989,0.640071,-0.889044,0.224857,-0.274063
5833,0.676543,0.0,0.0,0.040903,0.040903,0.0,0.017347,0.0,0.0,0.0,...,1.152986,0.140404,0.306716,-0.548324,0.373770,-0.424699,0.984228,-0.491309,0.491280,-0.400934


In [ ]:
prediction = model.predict(Yy)

In [ ]:
prediction.shape

(5835,)

In [ ]:
list(prediction)

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,


In [ ]:
import pandas as pd

# สร้าง DataFrame ตัวอย่าง
data = {
    'answer': list(prediction)
}

# สร้าง DataFrame โดยใช้ค่าดัชนีของแถวเป็นค่าในคอลัมน์ 'id'
submission_df = pd.DataFrame(data)
submission_df['id'] = submission_df.index # เพิ่ม 1 เพื่อให้ค่าดัชนีเริ่มจาก 1 ไม่ใช่ 0

# สลับตำแหน่งของคอลัมน์ 'id' ให้อยู่ด้านหน้า
cols = ['id', 'answer']
submission_df = submission_df[cols]

submission_df

,id,answer
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
5830,5830,0
5831,5831,0
5832,5832,1
5833,5833,0


In [ ]:
submission_df.to_csv('fusion_mLUKE.csv',index=False)

# Pycarret

In [ ]:
from pycaret.classification import *
from pycaret.clustering import *

In [ ]:
X_train_tfidf_df

,,,,',"',","','",'],(,(1),(2),...,ไธสง,ไพบูลย์,ไฟฟ้า,ไมเคิล,ไม่,ไม่ต้อง,ไว้,ๆ,่,y
0,0.318807,0.0,0.0,0.000000,0.000000,0.0,0.079702,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,0.301505,0.0,0.0,0.000000,0.000000,0.0,0.075376,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.241174,0.0,0.0,0.000000,0.000000,0.0,0.080391,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,0.300333,0.0,0.0,0.000000,0.000000,0.0,0.075083,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,0.253183,0.0,0.0,0.000000,0.000000,0.0,0.084394,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4424,0.540891,0.0,0.0,0.058763,0.058763,0.0,0.024586,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4425,0.576415,0.0,0.0,0.059899,0.059899,0.0,0.025062,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4426,0.555971,0.0,0.0,0.057775,0.057775,0.0,0.024173,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4427,0.570623,0.0,0.0,0.059297,0.059297,0.0,0.024810,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [ ]:
y

0       0
1       0
2       0
3       0
4       0
       ..
4424    1
4425    1
4426    0
4427    1
4428    0
Name: answer, Length: 4429, dtype: int64

In [ ]:
pce_1 = setup(data = train, target = 'answer', session_id = 21, train_size = 0.9)

,Description,Value
0,Session id,21
1,Target,answer
2,Target type,Binary
3,Original data shape,"(4429, 4)"
4,Transformed data shape,"(4429, 4)"
5,Transformed train set shape,"(3986, 4)"
6,Transformed test set shape,"(443, 4)"
7,Categorical features,3
8,Rows with missing values,0.0%
9,Preprocess,True


In [ ]:
best = compare_models(sort='F1')


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.7905,0.8771,0.4595,0.8236,0.5889,0.4635,0.4994,0.0700
nb,Naive Bayes,0.7632,0.8378,0.3745,0.7941,0.5068,0.3775,0.4242,0.0690
qda,Quadratic Discriminant Analysis,0.7582,0.8367,0.3585,0.7886,0.4904,0.3611,0.4100,0.0680
ada,Ada Boost Classifier,0.7536,0.8441,0.3307,0.7975,0.4615,0.3392,0.3953,0.1640
knn,K Neighbors Classifier,0.7406,0.7715,0.3292,0.7308,0.4500,0.3131,0.3566,0.0940
et,Extra Trees Classifier,0.7293,0.7906,0.3094,0.6832,0.4191,0.2801,0.3175,0.2060
dt,Decision Tree Classifier,0.7215,0.6176,0.3140,0.6433,0.4140,0.2660,0.2956,0.0690
gbc,Gradient Boosting Classifier,0.7373,0.8405,0.2963,0.7330,0.4103,0.2885,0.3377,0.2350
rf,Random Forest Classifier,0.7313,0.8290,0.2964,0.7066,0.4094,0.2783,0.3224,0.2400
lightgbm,Light Gradient Boosting Machine,0.7243,0.8257,0.2780,0.6864,0.3893,0.2567,0.2999,0.6090


Processing:   0%|          | 0/69 [00:00<?, ?it/s]

In [ ]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [ ]:
test['concat']

0       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
1       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
2       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
3       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
4       ลายมือชื่อของกรรมการคนใดคนหนึ่งลงลายมือชื่อ เป...
                              ...                        
5830    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5831    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5832    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5833    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
5834    กรรมการคนใดคนหนึ่งของกรรมการกลุ่ม ก. ลงลายมือช...
Name: concat, Length: 5835, dtype: object

In [ ]:
X_test_tfidf_df

,,,,',"',","','",'],(,(1),(2),...,ไทย,ไธสง,ไพบูลย์,ไฟฟ้า,ไมเคิล,ไม่,ไม่ต้อง,ไว้,ๆ,่
0,0.305278,0.0,0.0,0.000000,0.000000,0.0,0.061056,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.328008,0.0,0.0,0.000000,0.000000,0.0,0.065602,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.278466,0.0,0.0,0.000000,0.000000,0.0,0.055693,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.305180,0.0,0.0,0.000000,0.000000,0.0,0.061036,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.317961,0.0,0.0,0.000000,0.000000,0.0,0.063592,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5830,0.574025,0.0,0.0,0.036104,0.036104,0.0,0.015106,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5831,0.578071,0.0,0.0,0.036359,0.036359,0.0,0.015212,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5832,0.602526,0.0,0.0,0.036925,0.036925,0.0,0.015449,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5833,0.591172,0.0,0.0,0.036230,0.036230,0.0,0.015158,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
predictions_new_data = predict_model(best, data=X_test_tfidf_df)

In [ ]:
predictions_new_data

,,,,',"',","','",'],(,(1),(2),...,ไพบูลย์,ไฟฟ้า,ไมเคิล,ไม่,ไม่ต้อง,ไว้,ๆ,่,prediction_label,prediction_score
0,0.305278,0.0,0.0,0.000000,0.000000,0.0,0.061056,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.95
1,0.328008,0.0,0.0,0.000000,0.000000,0.0,0.065602,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.99
2,0.278466,0.0,0.0,0.000000,0.000000,0.0,0.055693,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.83
3,0.305180,0.0,0.0,0.000000,0.000000,0.0,0.061036,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.98
4,0.317961,0.0,0.0,0.000000,0.000000,0.0,0.063592,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5830,0.574025,0.0,0.0,0.036104,0.036104,0.0,0.015106,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.65
5831,0.578071,0.0,0.0,0.036359,0.036359,0.0,0.015212,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.58
5832,0.602526,0.0,0.0,0.036925,0.036925,0.0,0.015449,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.58
5833,0.591172,0.0,0.0,0.036230,0.036230,0.0,0.015158,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.59


In [ ]:
submission = predictions_new_data['prediction_label']
submission

0       1
1       1
2       1
3       1
4       1
       ..
5830    0
5831    0
5832    0
5833    0
5834    0
Name: prediction_label, Length: 5835, dtype: int64

In [ ]:
import pandas as pd

# สร้าง DataFrame ตัวอย่าง
data = {
    'answer': submission.to_list()
}

# สร้าง DataFrame โดยใช้ค่าดัชนีของแถวเป็นค่าในคอลัมน์ 'id'
submission_df = pd.DataFrame(data)
submission_df['id'] = submission_df.index # เพิ่ม 1 เพื่อให้ค่าดัชนีเริ่มจาก 1 ไม่ใช่ 0

# สลับตำแหน่งของคอลัมน์ 'id' ให้อยู่ด้านหน้า
cols = ['id', 'answer']
submission_df = submission_df[cols]

submission_df


,id,answer
0,0,1
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
5830,5830,0
5831,5831,0
5832,5832,0
5833,5833,0


In [ ]:
submission_df.to_csv('ensemble.csv',index=False)